# Imports

In [19]:
from problems.bin_packing_1d.definitions import *
from problems.bin_packing_1d.heuristics import *

from MCTS import *
from nested_methods import nmcs, nrpa
from ranked_rewards import R2

import random

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# Sanity checks

In [20]:
bin_size = 10
n_items = 25
max_item_weight = 4

In [21]:
def generate_items(n_items, max_item_weight):
    items = []
    for i in range(n_items):
        items.append(Item(i, random.randint(1, max_item_weight)))
    return items

In [22]:
items = generate_items(n_items, max_item_weight)

In [23]:
solution = first_fit(items, bin_size)
show_solution(items, solution, bin_size)

Items:  [Item(id=0, weight=3), Item(id=1, weight=4), Item(id=2, weight=3), Item(id=3, weight=2), Item(id=4, weight=4), Item(id=5, weight=2), Item(id=6, weight=3), Item(id=7, weight=2), Item(id=8, weight=1), Item(id=9, weight=4), Item(id=10, weight=3), Item(id=11, weight=2), Item(id=12, weight=1), Item(id=13, weight=2), Item(id=14, weight=2), Item(id=15, weight=4), Item(id=16, weight=1), Item(id=17, weight=1), Item(id=18, weight=1), Item(id=19, weight=3), Item(id=20, weight=3), Item(id=21, weight=1), Item(id=22, weight=3), Item(id=23, weight=2), Item(id=24, weight=1)]
Trivial lower bound:  6
Solution length:  9


In [24]:
# test
n_simulations = 10
state_0 = State([], bin_size, items, n_items)
node_0 = Node(state_0)

mcts_solver = MCTS(node_0)
next_state = mcts_solver.best_successor(n_simulations)

while not next_state.is_terminal():
    mcts_solver = MCTS(next_state)
    next_state = mcts_solver.best_successor(n_simulations)
    
show_solution(items, next_state.state.bins, bin_size)

Items:  [Item(id=0, weight=3), Item(id=1, weight=4), Item(id=2, weight=3), Item(id=3, weight=2), Item(id=4, weight=4), Item(id=5, weight=2), Item(id=6, weight=3), Item(id=7, weight=2), Item(id=8, weight=1), Item(id=9, weight=4), Item(id=10, weight=3), Item(id=11, weight=2), Item(id=12, weight=1), Item(id=13, weight=2), Item(id=14, weight=2), Item(id=15, weight=4), Item(id=16, weight=1), Item(id=17, weight=1), Item(id=18, weight=1), Item(id=19, weight=3), Item(id=20, weight=3), Item(id=21, weight=1), Item(id=22, weight=3), Item(id=23, weight=2), Item(id=24, weight=1)]
Trivial lower bound:  6
Solution length:  7


In [25]:
# test
state_0 = State([], bin_size, items, n_items)
lower_bound = math.ceil(sum([item.weight for item in items]) / bin_size)
sequence, score = nmcs(state_0, 1, - lower_bound)

state_1 = state_0
for s in sequence:
    state_1 = state_1.get_successor(s)

show_solution(items, state_1.bins, bin_size)

Items:  [Item(id=0, weight=3), Item(id=1, weight=4), Item(id=2, weight=3), Item(id=3, weight=2), Item(id=4, weight=4), Item(id=5, weight=2), Item(id=6, weight=3), Item(id=7, weight=2), Item(id=8, weight=1), Item(id=9, weight=4), Item(id=10, weight=3), Item(id=11, weight=2), Item(id=12, weight=1), Item(id=13, weight=2), Item(id=14, weight=2), Item(id=15, weight=4), Item(id=16, weight=1), Item(id=17, weight=1), Item(id=18, weight=1), Item(id=19, weight=3), Item(id=20, weight=3), Item(id=21, weight=1), Item(id=22, weight=3), Item(id=23, weight=2), Item(id=24, weight=1)]
Trivial lower bound:  6
Solution length:  6


In [26]:
# test
state_0 = State([], bin_size, items, n_items)
lower_bound = math.ceil(sum([item.weight for item in items]) / bin_size)
policy = dict()
sequence, score = nrpa(state_0, policy, 1, -lower_bound)

state_1 = state_0
for s in sequence:
    state_1 = state_1.get_successor(s)

show_solution(items, state_1.bins, bin_size)

Items:  [Item(id=0, weight=3), Item(id=1, weight=4), Item(id=2, weight=3), Item(id=3, weight=2), Item(id=4, weight=4), Item(id=5, weight=2), Item(id=6, weight=3), Item(id=7, weight=2), Item(id=8, weight=1), Item(id=9, weight=4), Item(id=10, weight=3), Item(id=11, weight=2), Item(id=12, weight=1), Item(id=13, weight=2), Item(id=14, weight=2), Item(id=15, weight=4), Item(id=16, weight=1), Item(id=17, weight=1), Item(id=18, weight=1), Item(id=19, weight=3), Item(id=20, weight=3), Item(id=21, weight=1), Item(id=22, weight=3), Item(id=23, weight=2), Item(id=24, weight=1)]
Trivial lower bound:  6
Solution length:  6


In [27]:
# simple CNN architecture
def create_model(bin_size, n_items):
    inputs = tf.keras.Input(shape=(n_items, bin_size, 2))
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    policy_head = layers.Conv2D(1, 3, activation='relu', padding='same')(x)
    policy_head = layers.Flatten()(policy_head)
    policy_head = layers.Dense(n_items * n_items, activation='softmax', name='policy')(policy_head)
    value_head = layers.Flatten()(x)
    value_head = layers.Dense(1, activation='tanh', name='value')(value_head)

    model = tf.keras.Model(inputs=inputs, outputs=[policy_head, value_head])
    return model

In [ ]:
model = create_model(bin_size, n_items)
model.compile(optimizer='adam',
              loss={'policy': 'sparse_categorical_crossentropy', 'value': 'mse'}, metrics=['accuracy'])

r2 = R2(nn=model)


instances_path = '/content/drive/MyDrive/MC_project/Scholl_1'

n_items = 50
bin_size = 100

training_instances = []

for file in os.listdir(instances_path):
    if not file.endswith('.p') and file.startswith('N1C1'):
        nb_items, bin_size, items = read_scholl_instance(os.path.join(instances_path, file))
        if nb_items == n_items:
            training_instances.append(items)

r2.r2(training_instances[:-5])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
1/1 [==============================] - 1s 766ms/step - loss: 8.8283 - policy_loss: 7.8335 - value_loss: 0.9948 - policy_accuracy: 0.0000e+00 - value_accuracy: 0.0000e+00 - val_loss: 9.7744 - val_policy_loss: 7.8161 - val_value_loss: 1.9583 - val_policy_accuracy: 0.0000e+00 - val_value_accuracy: 0.5104

Epoch 00001: val_loss improved from inf to 9.77440, saving model to /content/drive/MyDrive/MC_project/models/model_0.h5
Epoch 2/10
1/1 [==============================] - 0s 57ms/step - loss: 8.0576 - policy_loss: 7.8076 - value_loss: 0.2500 - policy_accuracy: 0.0000e+00 - value_accuracy: 0.9375 - val_loss: 9.8193 - val_policy_loss: 7.7776 - val_value_loss: 2.0417 - val_policy_accuracy: 0.0104 - val_value_accuracy: 0.4896

Epoch 00002: val_loss did not improve from 9.77440
Epoch 3/10
1/1 [==============================] - 0s 54ms/step - loss: 8.1382 - policy_loss: 7.7632 - value_loss: 0.3750 - policy_accuracy: 0.0000e+00 - value_accuracy: 0.9062 - val_loss: 9.6967 - val_policy_

In [ ]:
# test
model = tf.keras.models.load_model('models/model_0.h5')
n_simulations = 10


state_0 = State([], training_instances[-1])
node_0 = Node(state_0)

mcts_solver = MCTS(node_0, nn=model)
next_state = mcts_solver.best_successor(n_simulations)

while not next_state.is_terminal():
    mcts_solver = MCTS(next_state, nn=model)
    next_state = mcts_solver.best_successor(n_simulations)
    
show_solution(items, next_state.state.bins)

Items:  [Item(id=0, weight=100), Item(id=1, weight=99), Item(id=2, weight=98), Item(id=3, weight=97), Item(id=4, weight=95), Item(id=5, weight=94), Item(id=6, weight=93), Item(id=7, weight=93), Item(id=8, weight=93), Item(id=9, weight=92), Item(id=10, weight=92), Item(id=11, weight=92), Item(id=12, weight=92), Item(id=13, weight=85), Item(id=14, weight=85), Item(id=15, weight=83), Item(id=16, weight=81), Item(id=17, weight=79), Item(id=18, weight=77), Item(id=19, weight=76), Item(id=20, weight=75), Item(id=21, weight=73), Item(id=22, weight=71), Item(id=23, weight=70), Item(id=24, weight=70), Item(id=25, weight=69), Item(id=26, weight=66), Item(id=27, weight=63), Item(id=28, weight=60), Item(id=29, weight=60), Item(id=30, weight=59), Item(id=31, weight=59), Item(id=32, weight=58), Item(id=33, weight=58), Item(id=34, weight=57), Item(id=35, weight=49), Item(id=36, weight=48), Item(id=37, weight=47), Item(id=38, weight=45), Item(id=39, weight=42), Item(id=40, weight=41), Item(id=41, weig

In [ ]:
# test
model = tf.keras.models.load_model('models/model_9.h5')


state_0 = State([], training_instances[-1])
node_0 = Node(state_0)

mcts_solver = MCTS(node_0, nn=model)
next_state = mcts_solver.best_successor(n_simulations)

while not next_state.is_terminal():
    mcts_solver = MCTS(next_state, nn=model)
    next_state = mcts_solver.best_successor(n_simulations)
    
show_solution(items, next_state.state.bins)

Items:  [Item(id=0, weight=100), Item(id=1, weight=99), Item(id=2, weight=98), Item(id=3, weight=97), Item(id=4, weight=95), Item(id=5, weight=94), Item(id=6, weight=93), Item(id=7, weight=93), Item(id=8, weight=93), Item(id=9, weight=92), Item(id=10, weight=92), Item(id=11, weight=92), Item(id=12, weight=92), Item(id=13, weight=85), Item(id=14, weight=85), Item(id=15, weight=83), Item(id=16, weight=81), Item(id=17, weight=79), Item(id=18, weight=77), Item(id=19, weight=76), Item(id=20, weight=75), Item(id=21, weight=73), Item(id=22, weight=71), Item(id=23, weight=70), Item(id=24, weight=70), Item(id=25, weight=69), Item(id=26, weight=66), Item(id=27, weight=63), Item(id=28, weight=60), Item(id=29, weight=60), Item(id=30, weight=59), Item(id=31, weight=59), Item(id=32, weight=58), Item(id=33, weight=58), Item(id=34, weight=57), Item(id=35, weight=49), Item(id=36, weight=48), Item(id=37, weight=47), Item(id=38, weight=45), Item(id=39, weight=42), Item(id=40, weight=41), Item(id=41, weig

Took way too much time for not so good results, but might be interesting in the long run with other problems..

# Benchmarks

Experimenting with Scholl instances (http://or.dei.unibo.it/library/bpplib) with 50 items and a bin size of 100

In [ ]:
def read_instance(f_instance):
    items = []
    with open(f_instance, 'r') as f:
        n_items = int(next(f))
        bin_size = int(next(f))

        id_item = 0
        while True:
            try:
                line = next(f)
                items.append(Item(id_item, int(line)))
                id_item += 1
            except:
                break
    return n_items, bin_size, items

In [ ]:
instances_path = 'problems/bin_packing_1d/Scholl_1'

n_items = 50
bin_size = 100

instances_50_100 = []
instances_50_100_names = []

for file in os.listdir(instances_path):
    if file.startswith('N1C1'):
        instances_50_100.append(read_instance(os.path.join(instances_path, file))[2])
        instances_50_100_names.append(file)

## Comparing methods
Here we compare Heuristics, MCTS and variants, NMCS, and NRPA.
Transposition table does not appear here because it doesn't seem to improve results on this problem. And R2 has a prohibitive execution time

In [ ]:
solutions = []
exec_times = []

for i in range(10):
    solutions.append(dict())
    exec_times.append(dict())

    solutions[i]['instance_name'] = instances_50_100_names[i]
    exec_times[i]['instance_name'] = instances_50_100_names[i]

    solutions[i]['lower_bound'] = math.ceil(sum([item.weight for item in instances_50_100[i]]) / bin_size)

    # Heuristics
    t1 = time.time()
    solutions[i]['first_fit'] = len(first_fit(instances_50_100[i]))
    exec_times[i]['first_fit'] = time.time() - t1
    
    t1 = time.time()
    solutions[i]['best_fit'] = len(best_fit(instances_50_100[i]))
    exec_times[i]['best_fit'] = time.time() - t1

    t1 = time.time()
    solutions[i]['worst_fit'] = len(worst_fit(instances_50_100[i]))
    exec_times[i]['worst_fit'] = time.time() - t1

    t1 = time.time()
    solutions[i]['first_fit_decreasing'] = len(first_fit_decreasing(instances_50_100[i]))
    exec_times[i]['first_fit_decreasing'] = time.time() - t1
    
    t1 = time.time()
    solutions[i]['best_fit_decreasing'] = len(best_fit_decreasing(instances_50_100[i]))
    exec_times[i]['best_fit_decreasing'] = time.time() - t1

    t1 = time.time()
    solutions[i]['worst_fit_decreasing'] = len(worst_fit_decreasing(instances_50_100[i]))
    exec_times[i]['worst_fit_decreasing'] = time.time() - t1


    # MCTS
    simulations = [10, 20, 50, 100, 150]
    for n_simulations in simulations:
        
        state_0 = State([], instances_50_100[i])
        node_0 = Node(state_0)

        t1 = time.time()
        mcts_solver = MCTS(node_0)
        next_state = mcts_solver.best_successor(n_simulations)

        while not next_state.is_terminal():
            mcts_solver = MCTS(next_state)
            next_state = mcts_solver.best_successor(n_simulations)
            
        solutions[i]['MCTS_' + str(n_simulations)] = len(next_state.state.bins)
        exec_times[i]['MCTS_' + str(n_simulations)] = time.time() - t1

    levels = [1]

    for level in levels:

        # NMCS
        state_0 = State([], instances_50_100[i])
        t1 = time.time()
        sequence, score = nmcs(state_0, level, - solutions[i]['lower_bound'])
        exec_times[i]['NMCS_' + str(level)] = time.time() - t1

        state_1 = state_0
        for s in sequence:
            state_1 = state_1.get_successor(s)

        solutions[i]['NMCS_' + str(level)] = len(state_1.bins)

        # NRPA
        policy = dict()
        t1 = time.time()
        sequence, score = nrpa(state_0, policy, level, - solutions[i]['lower_bound'])
        exec_times[i]['NRPA_' + str(level)] = time.time() - t1

        state_1 = state_0
        for s in sequence:
            state_1 = state_1.get_successor(s)

        solutions[i]['NRPA_' + str(level)] = len(state_1.bins)

    #if i%10 == 0:
    print(i + 1, 'instances processed..')

4 instances processed..
5 instances processed..
6 instances processed..
7 instances processed..
8 instances processed..
9 instances processed..
10 instances processed..


In [ ]:
#pickle.dump(solutions, open( os.path.join(instances_path, 'results_scholl_1_10.p'), "wb"))
#pickle.dump(exec_times, open( os.path.join(instances_path, 'exec_times_scholl_1_10.p'), "wb"))

In [ ]:
df_solutions = pd.DataFrame(solutions)
df_exec_times = pd.DataFrame(exec_times)

In [ ]:
df_solutions

,instance_name,lower_bound,first_fit,best_fit,worst_fit,first_fit_decreasing,best_fit_decreasing,worst_fit_decreasing,MCTS_10,MCTS_20,MCTS_50,MCTS_100,MCTS_150,NMCS_1,NRPA_1
0,N1C1W1_B.txt,28,34,31,31,34,31,31,33,34,34,33,32,32,31
1,N1C1W2_S.txt,33,39,37,37,39,37,37,40,40,39,39,38,37,38
2,N1C1W2_T.txt,33,42,38,38,42,38,38,39,39,39,39,39,38,38
3,N1C1W4_Q.txt,31,37,34,34,37,34,34,38,38,37,37,37,35,35
4,N1C1W1_G.txt,25,30,25,26,30,25,26,30,30,29,29,28,27,28
5,N1C1W1_Q.txt,26,29,28,28,29,28,28,32,32,31,30,30,29,30
6,N1C1W1_R.txt,24,28,25,25,28,25,25,28,29,28,27,27,27,27
7,N1C1W1_C.txt,20,24,21,21,24,21,21,25,24,24,23,24,22,22
8,N1C1W4_M.txt,33,44,41,41,44,41,41,43,42,42,41,41,41,41
9,N1C1W2_N.txt,29,36,33,33,36,33,33,37,38,35,36,34,34,34


In [ ]:
df_exec_times

,instance_name,first_fit,best_fit,worst_fit,first_fit_decreasing,best_fit_decreasing,worst_fit_decreasing,MCTS_10,MCTS_20,MCTS_50,MCTS_100,MCTS_150,NMCS_1,NRPA_1
0,N1C1W1_B.txt,0.000230,0.000182,0.000194,0.000239,0.000186,0.000195,1.499926,3.601065,9.382067,27.680643,93.590647,8.529761,4.310503
1,N1C1W2_S.txt,0.000282,0.000191,0.000194,0.000284,0.000203,0.000200,1.629910,3.496780,10.209424,31.079728,86.729661,6.903998,3.845705
2,N1C1W2_T.txt,0.000299,0.000228,0.000196,0.000289,0.000205,0.000224,1.607088,3.573178,10.723592,39.953628,84.533070,8.361970,4.267251
3,N1C1W4_Q.txt,0.000337,0.000187,0.000182,0.000264,0.000186,0.000197,1.608448,3.716609,9.854492,42.636587,82.291520,10.577245,4.138575
4,N1C1W1_G.txt,0.000219,0.000161,0.000166,0.000214,0.000171,0.000171,2.162745,3.783035,9.863304,25.459381,76.696203,11.451276,5.584491
5,N1C1W1_Q.txt,0.000203,0.000173,0.000179,0.000211,0.000206,0.000185,1.524879,3.595890,9.120170,25.682026,86.600226,10.109490,4.566962
6,N1C1W1_R.txt,0.000215,0.000161,0.000165,0.000244,0.000168,0.000178,1.457466,3.708254,9.338645,41.791189,67.554100,12.579347,5.589445
7,N1C1W1_C.txt,0.000190,0.000143,0.000171,0.000180,0.000156,0.000151,1.680046,4.353373,9.900656,22.422966,53.870395,18.722876,6.639087
8,N1C1W4_M.txt,0.000318,0.000197,0.000197,0.000313,0.000210,0.000203,2.178362,3.398255,11.444605,43.612464,81.574830,7.236372,4.033579
9,N1C1W2_N.txt,0.000253,0.000181,0.000194,0.000261,0.000187,0.000194,1.801389,3.329090,9.298556,30.897008,91.894804,8.944566,4.782875


**Summary**
Entries in bold mean that the algorithm has failed to find the optimal solution

| Instance name | Optimal (http://or.dei.unibo.it/library/bpplib) | Heuristics | MCTS     | NMCS | NRPA |
|---------------|-------------------------------------------------|------------|----------|------|------|
| N1C1W1_B      | 31                                              | 31         | **32** (150) | **32**   | 31   |
| N1C1W2_S      | 37                                              | 37         | **38** (150) | 37   | **38**   |
| N1C1W2_T      | 38                                              | 38         | **39** (10)  | 38   | 38   |
| N1C1W4_Q      | 34                                              | 34         | **37**(150)  | **35**   | **35**   |
| N1C1W1_G      | 25                                              | 25         | **28** (150) | **27**   | **28**   |
| N1C1W1_Q      | 28                                              | 28         | **30** (100) | **29**   | **30**   |
| N1C1W1_R      | 25                                              | 25         | **27** (100) | **27**   | **27**   |
| N1C1W1_C      | 20                                              | **21**         | **23** (100) | **22**   | **22**   |
| N1C1W4_M      | 41                                              | 41         | 41 (100) | 41   | 41   |
| N1C1W2_N      | 33                                              | 33         | **34** (150) | **34**   | **34**   |


Hard 28 dataset: These instances are supposed to be harder than Scholl. For timing reasons, we were only able to test on 5 instances.

In [ ]:
instances_path = 'problems/bin_packing_1d/Hard28'

n_items = 200
bin_size = 1000

instances_200_1000 = []
instances_200_1000_names = []

for file in os.listdir(instances_path):
    nb_items, bin_size, items = read_instance(os.path.join(instances_path, file))
    if nb_items == n_items:
        instances_200_1000.append(items)
        instances_200_1000_names.append(file)

Compare methods

In [ ]:
solutions = []
exec_times = []

for i in range(len(instances_50_100)):
    solutions.append(dict())
    exec_times.append(dict())

    solutions[i]['instance_name'] = instances_200_1000_names[i]
    exec_times[i]['instance_name'] = instances_200_1000_names[i]

    solutions[i]['lower_bound'] = math.ceil(sum([item.weight for item in instances_200_1000[i]]) / bin_size)

    # Heuristics
    t1 = time.time()
    solutions[i]['first_fit'] = len(first_fit(instances_200_1000[i]))
    exec_times[i]['first_fit'] = time.time() - t1
    
    t1 = time.time()
    solutions[i]['best_fit'] = len(best_fit(instances_200_1000[i]))
    exec_times[i]['best_fit'] = time.time() - t1

    t1 = time.time()
    solutions[i]['worst_fit'] = len(worst_fit(instances_200_1000[i]))
    exec_times[i]['worst_fit'] = time.time() - t1

    t1 = time.time()
    solutions[i]['first_fit_decreasing'] = len(first_fit_decreasing(instances_200_1000[i]))
    exec_times[i]['first_fit_decreasing'] = time.time() - t1
    
    t1 = time.time()
    solutions[i]['best_fit_decreasing'] = len(best_fit_decreasing(instances_200_1000[i]))
    exec_times[i]['best_fit_decreasing'] = time.time() - t1

    t1 = time.time()
    solutions[i]['worst_fit_decreasing'] = len(worst_fit_decreasing(instances_200_1000[i]))
    exec_times[i]['worst_fit_decreasing'] = time.time() - t1


    # MCTS
    simulations = [10, 20, 50, 100]
    for n_simulations in simulations:
        
        state_0 = State([], instances_200_1000[i])
        node_0 = Node(state_0)

        t1 = time.time()
        mcts_solver = MCTS(node_0)
        next_state = mcts_solver.best_successor(n_simulations)

        while not next_state.is_terminal():
            mcts_solver = MCTS(next_state)
            next_state = mcts_solver.best_successor(n_simulations)
            
        solutions[i]['MCTS_' + str(n_simulations)] = len(next_state.state.bins)
        exec_times[i]['MCTS_' + str(n_simulations)] = time.time() - t1

    levels = [1]

    for level in levels:

        # NMCS
        state_0 = State([], instances_200_1000[i])
        t1 = time.time()
        sequence, score = nmcs(state_0, level, - solutions[i]['lower_bound'])
        exec_times[i]['NMCS_' + str(level)] = time.time() - t1

        state_1 = state_0
        for s in sequence:
            state_1 = state_1.get_successor(s)

        solutions[i]['NMCS_' + str(level)] = len(state_1.bins)

        # NRPA
        policy = dict()
        t1 = time.time()
        sequence, score = nrpa(state_0, policy, level, - solutions[i]['lower_bound'])
        exec_times[i]['NRPA_' + str(level)] = time.time() - t1

        state_1 = state_0
        for s in sequence:
            state_1 = state_1.get_successor(s)

        solutions[i]['NRPA_' + str(level)] = len(state_1.bins)

    if i%10 == 0:
        print(i + 1, 'instances processed..')

In [ ]:
df_solutions = pd.DataFrame(solutions)
df_exec_times = pd.DataFrame(exec_times)

In [ ]:
df_solutions

,instance_name,lower_bound,first_fit,best_fit,worst_fit,first_fit_decreasing,best_fit_decreasing,worst_fit_decreasing,MCTS_10,MCTS_20,MCTS_50,MCTS_100,NMCS_1,NRPA_1
0,Hard28_BPP144.txt,73,80,74,74,80,74,74,100,100,99,94,79,82
1,Hard28_BPP119.txt,76,83,77,78,83,77,78,105,105,104,99,94,96
2,Hard28_BPP814.txt,81,86,82,84,86,82,84,109,110,109,106,90,93
3,Hard28_BPP561.txt,72,80,73,74,80,73,74,95,96,96,94,77,80
4,Hard28_BPP531.txt,83,88,84,85,88,84,85,115,114,115,109,94,97


In [ ]:
df_exec_times

,instance_name,first_fit,best_fit,worst_fit,first_fit_decreasing,best_fit_decreasing,worst_fit_decreasing,MCTS_10,MCTS_20,MCTS_50,MCTS_100,NMCS_1,NRPA_1
0,Hard28_BPP144.txt,0.002393,0.001767,0.001910,0.002373,0.001554,0.001614,149.713561,292.477625,718.060937,1463.953218,6394.303658,125.533923
1,Hard28_BPP119.txt,0.002037,0.001643,0.001677,0.002002,0.001649,0.001696,146.689051,268.661875,694.443333,1405.011178,5766.494354,121.961105
2,Hard28_BPP814.txt,0.001882,0.001755,0.001805,0.001895,0.001760,0.001850,138.614182,270.273484,678.834056,1454.310308,5014.162036,124.290952
3,Hard28_BPP561.txt,0.002001,0.001582,0.001523,0.002051,0.001573,0.001579,158.446825,282.897628,716.459893,1481.062137,6647.424275,140.297726
4,Hard28_BPP531.txt,0.001871,0.001750,0.001803,0.001989,0.001814,0.001977,143.326707,281.967380,720.171445,1471.481190,5103.881170,128.329832


## Combining Heuristics with MC methods

From previous experiments, we notice that best fit heuristic works very well on the problem. As a result, we try here to use MCTS to improve its results on some harder instances" (We use Scholl dataset since it is much faster, and according to our experiments, best fit only creates one extra bin in both datasets)

In [ ]:
df_solutions = pd.DataFrame(solutions)

In [ ]:
df_solutions

,instance_name,lower_bound,first_fit,best_fit,worst_fit,first_fit_decreasing,best_fit_decreasing,worst_fit_decreasing,MCTS_10,MCTS_20,MCTS_50,MCTS_100,MCTS_150,NMCS_1,NRPA_1
0,N1C1W2_B.txt,29,34,30,30,34,30,30,34,35,33,33,32,31,31
1,N1C1W1_K.txt,24,28,26,26,28,26,26,31,29,28,28,28,27,27
2,N1C1W2_P.txt,30,35,33,33,35,33,33,37,37,36,35,35,34,34
3,N1C1W4_P.txt,33,40,38,38,40,38,38,41,41,39,39,39,38,39
4,N1C1W1_M.txt,28,32,30,30,32,30,30,34,32,31,31,32,30,30
5,N1C1W4_E.txt,33,41,38,38,41,38,38,41,42,40,40,40,39,39
6,N1C1W4_A.txt,32,38,35,35,38,35,35,38,37,38,36,36,35,36
7,N1C1W2_L.txt,30,34,31,32,34,31,32,37,35,35,33,34,33,33
8,N1C1W2_D.txt,29,33,31,31,33,31,31,37,36,34,34,33,32,33
9,N1C1W1_G.txt,25,30,25,26,30,25,26,29,30,29,28,28,27,27


In [ ]:
solutions_df = df_solutions[['instance_name', 'best_fit']]

In [ ]:
df_optimal = pd.read_csv('results.csv')     # exracted from http://or.dei.unibo.it/sites/or.dei.unibo.it/files/instances/Solutions.rar

df_optimal.merge(solutions_df, left_on='Name', right_on='instance_name')

,Name,Best LB,Best UB,Status,Selected
0,N1C1W1_A.txt,25,25,Solved,0
1,N1C1W1_B.txt,31,31,Solved,0
2,N1C1W1_C.txt,20,20,Solved,1
3,N1C1W1_D.txt,28,28,Solved,0
4,N1C1W1_E.txt,26,26,Solved,0
...,...,...,...,...,...
1205,HARD5.txt,56,56,Solved,1
1206,HARD6.txt,57,57,Solved,1
1207,HARD7.txt,55,55,Solved,1
1208,HARD8.txt,57,57,Solved,1


In [ ]:
df_compare = df_optimal.merge(solutions_df, left_on='Name', right_on='instance_name')[['Name', 'Best LB', 'best_fit']]
df_compare['extra_bins'] = df_compare['best_fit'] - df_compare['Best LB']

In [ ]:
df_hard_instances = df_compare[df_compare['extra_bins'] > 0]

In [ ]:
#eps = [0.05, .1, .15, .2, 1]
eps = [.01]
n_simulations = 200

hard_instances_scholl = []
hard_instances_scholl_names = []

for n in list(df_hard_instances['Name']):
    hard_instances_scholl.append(read_instance(os.path.join(instances_path, n))[2])
    hard_instances_scholl_names.append(n)

for i, _ in enumerate(hard_instances_scholl):
    #solutions.append(dict())
    solutions[i]['Name'] = hard_instances_scholl_names[i]
    for e in eps:
        state_0 = State([], hard_instances_scholl[i])
        node_0 = Node(state_0)

        t1 = time.time()
        mcts_solver = MCTS(node_0, epsilon=e)
        next_state = mcts_solver.best_successor(n_simulations)

        while not next_state.is_terminal():
            mcts_solver = MCTS(next_state, epsilon=e)
            next_state = mcts_solver.best_successor(n_simulations)
            
        solutions[i]['MCTS_200_e' + str(e)] = len(next_state.state.bins)
    #exec_times[i]['MCTS_20_e' + str(e)] = time.time() - t1

In [ ]:
pd.DataFrame(solutions).merge(df_optimal[['Name', 'Best LB']], on='Name')

,Name,MCTS_200_e0.05,MCTS_200_e0.1,MCTS_200_e0.15,MCTS_200_e0.2,MCTS_200_e1,MCTS_200_e0.01,Best LB
0,N1C1W1_C.txt,21,21,21,21,23,21,20
1,N1C1W1_N.txt,25,26,26,27,28,25,25
2,N1C1W4_G.txt,37,38,38,38,38,37,37


Using MCTS, we were able to improve 2 instances over 3 (N1C1W1_N and N1C1W4_G)

Let's try to improve the hardest instance with NMCS (with best fit)

In [ ]:
n_items, bin_size, items_hard = read_instance('problems/bin_packing_1d/Scholl_1/N1C1W1_C.txt')
eps = [.01, .05, .1, .15, .2, 1]
solution_hard = dict()
lb = 20

solution_hard['Name'] = 'N1C1W1_C.txt'
for e in eps:
    print(e)
    state_0 = State([], items_hard)
    sequence, score = nmcs(state_0, 1, - lb, epsilon=e)

    state_1 = state_0
    for s in sequence:
        state_1 = state_1.get_successor(s)

    solution_hard['NMCS_' + str(level) + '_' + str(e)] = len(state_1.bins)

0.01
0.05
0.1
0.15
0.2
1


In [ ]:
pd.DataFrame([solution_hard])

,Name,NMCS_2_0.01,NMCS_2_0.05,NMCS_2_0.1,NMCS_2_0.15,NMCS_2_0.2,NMCS_2_1
0,N1C1W1_C.txt,22,22,22,22,22,22
